In [26]:
from fastai.tabular.all import *
from fastcore.utils import *
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import os

In [45]:
df = pd.read_csv(r'/home/kchen/Documents/AL/Data/data7.csv', index_col=0)
df.head()

,AGE,HEIGHT,WEIGHT,BMI,PRSODM,PRBUN,PRCREAT,PRALBUM,PRWBC,PRHCT,...,COL_APPROACH_9,COL_APPROACH_10,COL_APPROACH_11,OPERYR_0,OPERYR_1,OPERYR_2,OPERYR_3,OPERYR_4,OPERYR_5,OPERYR_6
CASEID,,,,,,,,,,,,,,,,,,,,,
8080841,0.698630,0.565217,0.220459,0.163738,0.560606,0.060302,0.032886,0.235294,0.062205,0.363462,...,0,0,0,0,0,0,0,0,0,1
8080850,0.452055,0.652174,0.250441,0.160112,0.500000,0.095477,0.059060,0.200000,0.029218,0.350000,...,0,0,0,0,0,0,0,0,0,1
8080874,0.164384,0.456522,0.292769,0.286363,0.545455,0.030151,0.027517,0.305882,0.101037,0.536538,...,0,0,0,0,0,0,0,0,0,1
8080921,0.657534,0.695652,0.294533,0.180788,0.515152,0.050251,0.061745,0.152941,0.074741,0.269231,...,0,0,0,0,0,0,0,0,0,1
8080981,0.712329,0.565217,0.213404,0.157164,0.590909,0.055276,0.046980,0.364706,0.095193,0.653846,...,0,0,0,0,0,0,0,0,0,1


In [46]:
df.shape

(146781, 128)

In [47]:
cont = ['AGE', 'HEIGHT', 'WEIGHT', 'BMI', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRWBC', 'PRHCT', 'PRPLATE']
cat = ['PRNCPTX', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'CONCPT1', 'COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_INDICATION', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'ASACLAS', 'TRANST','WNDCLAS', 'CPT', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'COL_APPROACH', 'OPERYR']

In [49]:
splits = RandomSplitter(seed=0)(df)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
procs = [Categorify, Normalize]
cont,cat = cont_cat_split(df, max_card=500, dep_var='COL_ANASTOMOTIC')
to_nn = TabularPandas(df, procs, cat, cont, splits=splits, y_names='COL_ANASTOMOTIC')
dls = to_nn.dataloaders(1024, device = device)

In [50]:
learn = tabular_learner(dls, layers=[500, 250, 250], n_out=1)
learn.fit_one_cycle(5, 1e-4)

epoch,train_loss,valid_loss,time
0,0.086988,0.051941,00:19
1,0.046233,0.039661,00:19
2,0.037888,0.036939,00:20
3,0.034910,0.035403,00:19
4,0.033372,0.035007,00:19


In [51]:
preds,targs = learn.get_preds()
roc_auc_score(targs, preds)

0.53363592101898

In [28]:
os.getcwd()

'/home/kchen/Documents/AL/Source'

In [52]:
learn.save('learn1')

Path('models/learn1.pth')

In [53]:
df1 = pd.read_csv(r'/home/kchen/Documents/AL/Data/data7.csv', low_memory=False, index_col=0)

In [21]:
procs = [Categorify]
to = TabularPandas(df1, procs, cat, cont, 'COL_ANASTOMOTIC', splits = splits)


In [37]:
def rf(xs, y, n_estimators=40, max_samples=100_000,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestClassifier(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf).fit(xs, y)

In [41]:
def auc(model, X, y):
    ppreds = model.predict_proba(X)
    ppreds = ppreds[:,1]
    pscore = roc_auc_score(y, ppreds)
    print('AUC', pscore)

In [31]:
learn = learn.load('learn1')

In [32]:
def embed_features(learner, xs):
  """
  learner: fastai Learner used to train the neural net
  xs: DataFrame containing input variables with nominal values defined by their rank.
  ::returns:: a copy of `xs` with embeddings replacing each categorical variable
  """
  xs = xs.copy()
  for i,col in enumerate(learn.dls.cat_names):
    emb = learn.model.embeds[i]
    emb_data = emb(tensor(xs[col], dtype=torch.int64).to(device))
    emb_names = [f'{col}_{j}' for j in range(emb_data.shape[1])]
    feat_df = pd.DataFrame(data=emb_data, index=xs.index, columns=emb_names)
    xs = xs.drop(col, axis=1)
    xs = xs.join(feat_df)
  return xs

In [33]:
emb_xs = embed_features(learn, to.train.xs)
emb_valid_xs = embed_features(learn, to.valid.xs)

In [42]:
m_cat = rf(to.train.xs, to.train.y)
auc(m_cat, to.train.xs, to.train.y), auc(m_cat, to.valid.xs, to.valid.y)

AUC 0.9993158674533451
AUC 0.5956933523388667


(None, None)

In [44]:
m_emb = rf(emb_xs, to.train.y)
auc(m_emb, emb_xs, to.train.y), auc(m_emb, emb_valid_xs, to.valid.y)

AUC 0.9988475227043866
AUC 0.6151324376669265


(None, None)

In [ ]:
# import modules, read data, and define options
df = pd.read_csv(r'/home/kchen/Documents/AL/Data/data3.csv', low_memory=False, index_col=0)
cont,cat = cont_cat_split(df, max_card=1000, dep_var='COL_ANASTOMOTIC')
procs = [Categorify]
splits = RandomSplitter()(df)
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
# feed the data into the Learner and train
to_nn = TabularPandas(nn, procs, cat, cont,
                      splits=splits, y_names='COL_ANASTOMOTIC')
dls = to_nn.dataloaders(1024, device = device)
learn = tabular_learner(dls, layers=[500,250], n_out=1)
learn.fit_one_cycle(12, 3e-3)